# Data Preparation: Preprocessing
Este notebook apresenta a etapa de preparação dos dados, contemplando os passos necessários do pré-processamento, úteis para o treinamento do modelo de reranker.

Neste momento, são então realizados:
- Limpeza completa e normalização textual
- Normalização do score de relevância
- Seleção das features necessárias

Ao final, os dados processados são salvos no formato `parquet` para utilização nas etapas futuras.

### **Observações quanto à etapa de Data Understanding**
Com base nas análises feitas na etapa anterior, observou-se que o Quati Dataset por si só apresenta poucos dados relevantes para o treinamento de um modelo de reranker neural.

Tendo isso em vista, optou-se por manter o **Quati Dataset como o conjunto de dados que define o problema de negócio** e foi acrescentado uma **nova base de dados suplementar especificamente para o treinamento do reranker**.

A base de dados selecionada foi o **MS MARCO**, um dataset robusto e padrão para treinar rerankers e retrievers no estado da arte. Ele possui uma estrutura semelhante ao Quati Dataset, uma quantidade massiva de dados, apesar de conter apenas textos em inglês.

Desta forma, se faz importante a seleção de features adequadas para convergir os dois datasets, a definição de um embedding multilinguístico para representação vetorial apurada e eficiente para os domínios linguísticos além de considerar análises qualitativas cross-linguísticas ao final do projeto para avaliar perfomance do reranker no contexto do domínio desenvolvido.

----

## Importações e instalações

In [1]:
# Instalações
!pip install "datasets<4.0.0"

In [2]:
# Imports
# Utils
import os
import warnings
cpu_count = os.cpu_count()
print(f"CPU count: {cpu_count}")
warnings.filterwarnings('ignore')

# Manipulação de dados
import numpy as np
import pandas as pd
from datasets import load_dataset, load_from_disk

# NLP
import re
import unicodedata

CPU count: 2


## Carregamento dos datasets

### Quati Dataset

In [3]:
# Load Quati topics
quati_topics = load_dataset("parquet", data_files="./cleaned_topics.parquet")["train"]

print(quati_topics)
print("-" * 100)
quati_topics[:5]

Dataset({
    features: ['query_id', 'query'],
    num_rows: 200
})
----------------------------------------------------------------------------------------------------


{'query_id': [0, 1, 2, 3, 4],
 'query': ['qual a maior característica da flora brasileira?',
  'qual a maior característica da fauna brasileira?',
  'por que os países guiana e suriname não são filiados a conmebol?',
  'por que a bolívia tem duas capitais?',
  'qual o que significa decolonialidade?']}

In [4]:
# Load Quati qrels
quati_qrels = load_dataset("parquet", data_files="./cleaned_qrels.parquet")["train"]

print(quati_qrels)
print("-" * 100)
quati_qrels[:5]

Dataset({
    features: ['query_id', 'passage_id', 'score'],
    num_rows: 1933
})
----------------------------------------------------------------------------------------------------


{'query_id': [1, 1, 1, 1, 1],
 'passage_id': ['clueweb22-pt0000-78-09747_0',
  'clueweb22-pt0000-96-07278_111',
  'clueweb22-pt0001-85-06153_3',
  'clueweb22-pt0000-64-06285_35',
  'clueweb22-pt0000-87-13049_0'],
 'score': [1, 1, 1, 2, 0]}

In [5]:
# Load Quati passages
quati_passages = load_dataset("parquet", data_files="./cleaned_passages.parquet")["train"]

print(quati_passages)
print("-" * 100)
quati_passages[:3]

Dataset({
    features: ['passage_id', 'passage'],
    num_rows: 1000000
})
----------------------------------------------------------------------------------------------------


{'passage_id': ['clueweb22-pt0000-00-00003_1',
  'clueweb22-pt0000-00-00003_2',
  'clueweb22-pt0000-00-00003_3'],
 'passage': ['se você precisar de ajuda, visite o website nacional sobre a covid-19 ou ligue para a linha de apoio à covid-19 808 24 24 24 perguntas mais frequentes posso viajar entre sintra e cascais? quais são as restrições de viagem em cascais? qual o número de telefone de apoio para a covid 19 em cascais? preciso utilizar máscara facial no transporte público em cascais? a prática do distanciamento social é compulsória em cascais? o que eu devo fazer caso apresente sintomas da covid-19 quando chegar em cascais? última atualização: 25 abr 2022 aplicam-se exceções, para detalhes completos: european union. estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à covid-19. esta informação é compilada a partir de fontes oficiais. ao melhor de nosso conhecimento, está correta de acordo com a última atualização. visite avisos de viag

### MS MARCO

In [6]:
# Load MS MARCO
msmarco = load_dataset("microsoft/ms_marco", "v2.1", split="train")

msmarco

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 808731
})

In [7]:
# Display some queries
msmarco["query"][:3]

[')what was the immediate impact of the success of the manhattan project?',
 '_________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:',
 'why did stalin want control of eastern europe']

In [8]:
# Display some passages
msmarco["passages"][0]

{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'passage_text': ['The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.',
  'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.',
  'Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.',
  'The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project

### Verificações básicas no MS MARCO
Tendo em vista que o MS MARCO não terá uma análise exploratória extensa, dado o seu uso suplementar, é necessário ao menos realizar uma verificação básica (textos nulos, quantidade e tamanho dos textos, distribuição dos scores) dos dados presentes para segurança e rigorosidade qualitativa dos dados.

#### Contagem de valores nulos

In [9]:
# Check for null values in each column
# Function to count null in batches
def count_nulls(batch):
    """
    Verifica a quantidade de valores nulos em cada coluna de um batch.
    """
    return {
        col: [x is None or x == "" for x in batch[col]]
        for col in batch
    }

# Null counts flags
null_flags = msmarco.map(
    count_nulls,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

# Count rows with null values
null_counts = {
    col: int(sum(null_flags[col]))
    for col in null_flags.column_names
}

# Display
for col, count in null_counts.items():
    print(f"Column '{col}': {count} null values")

Column 'answers': 0 null values
Column 'passages': 0 null values
Column 'query': 0 null values
Column 'query_id': 0 null values
Column 'query_type': 0 null values
Column 'wellFormedAnswers': 0 null values


#### Contagem de queries e documentos

In [10]:
# Count how many unique queries there are
print(f"Number of unique queries: {len(set(msmarco['query_id']))}")

Number of unique queries: 808731


In [11]:
# Count how many unique passages there are
# Function to count passages per query in batches
# Since each query has its dict with documents
def count_passages(batch):
    return {
        "num_passages": [len(p["passage_text"]) for p in batch["passages"]]
    }

# Passages count per query
passages_per_query = msmarco.map(
    count_passages,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

# Total passages count
total_passages_count = sum(passages_per_query["num_passages"])

# Display
print(f"Total number of passages: {total_passages_count}")
print(f"Min number of passages per query: {np.min(passages_per_query['num_passages'])}")
print(f"Mean number of passages per query: {np.mean(passages_per_query['num_passages'])}")
print(f"Median number of passages per query: {np.median(passages_per_query['num_passages'])}")
print(f"Max number of passages per query: {np.max(passages_per_query['num_passages'])}")

Total number of passages: 8069749
Min number of passages per query: 1
Mean number of passages per query: 9.978285734069797
Median number of passages per query: 10.0
Max number of passages per query: 27


#### Tamanho das queries e documentos (por palavras)

In [12]:
# Length of queries
def get_query_lens(batch):
    queries = batch["query"]
    return {
        "query_num_words": [len(q.split()) for q in queries]
    }

query_lens = msmarco.map(
    get_query_lens,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)
query_lens = np.array(query_lens["query_num_words"])

print(f"Min query length: {query_lens.min()}")
print(f"Mean query length: {query_lens.mean()}")
print(f"Median query length: {np.median(query_lens)}")
print(f"Max query length: {query_lens.max()}")

Min query length: 1
Mean query length: 6.365910296501556
Median query length: 6.0
Max query length: 75


In [13]:
# Length of passages
def passage_word_stats(batch):
    min_words = []
    mean_words = []
    median_words = []
    max_words = []

    for passages in batch["passages"]:
        counts = [len(p.split()) for p in passages["passage_text"]]
        if not counts:
            min_words.append(0) # Append 0 if no passages to avoid error later
            mean_words.append(0)
            median_words.append(0)
            max_words.append(0)
            continue

        min_words.append(min(counts))
        mean_words.append(sum(counts) / len(counts))
        median_words.append(np.median(counts))
        max_words.append(max(counts))

    return {
        "passage_min_words": min_words,
        "passage_mean_words": mean_words,
        "passage_median_words": median_words,
        "passage_max_words": max_words
    }

passage_lens = msmarco.map(
    passage_word_stats,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

# Convert the columns to numpy arrays before calculating statistics
min_passage_words = np.array(passage_lens['passage_min_words'])
mean_passage_words = np.array(passage_lens['passage_mean_words'])
median_passage_words = np.array(passage_lens['passage_median_words'])
max_passage_words = np.array(passage_lens['passage_max_words'])

print(f"Min passage word count: {np.min(min_passage_words)}")
print(f"Mean passage word count: {np.mean(mean_passage_words)}")
print(f"Median passage word count: {np.median(median_passage_words)}")
print(f"Max passage word count: {np.max(max_passage_words)}")

Min passage word count: 1
Mean passage word count: 56.47883927567174
Median passage word count: 49.0
Max passage word count: 362


#### Distribuição dos scores

In [14]:
# Function to count score distribution in batches
def count_is_selected(batch):
    count_0 = 0
    count_1 = 0

    for passages in batch["passages"]:
        for v in passages["is_selected"]:
            if v == 1:
                count_1 += 1
            else:
                count_0 += 1

    return {
        "count_0": [count_0],
        "count_1": [count_1],
    }

In [15]:
# Scores value count
score_counts = msmarco.map(
    count_is_selected,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count,
    remove_columns=msmarco.column_names
)

total_counts = {
    "count_0": sum(score_counts["count_0"]),
    "count_1": sum(score_counts["count_1"]),
    "proportion_0": score_counts["count_0"][0] / (score_counts["count_0"][0] + score_counts["count_1"][0]),
    "proportion_1": score_counts["count_1"][0] / (score_counts["count_0"][0] + score_counts["count_1"][0]),
}

# Display counts
print(f"Total count of 0: {total_counts['count_0']} ({total_counts['proportion_0'] * 100:.2f}%)")
print(f"Total count of 1: {total_counts['count_1']} ({total_counts['proportion_1'] * 100:.2f}%)")

Total count of 0: 7536988 (93.52%)
Total count of 1: 532761 (6.48%)


## Pré-processamento
Para o pré-processamento dos dados, é necessário limpar as features textuais de ambos os datasets e normalizar os scores de relevância (target), principalmente para o dataset Quati que possui valores entre 0 e 3.

Para a limpeza textual, é importante considerar e aplicar uma limpeza minimalista. Limpezas extensas e robustas podem diminuir a riqueza semântica dos textos, aumentar a ambiguidade, além de ser custoso computacionalmente. Dito isso, a limpeza textual se consiste principalmente na remoção de ruídos e padronização textual através de regex.

Para a normalização dos scores no Quati Dataset, os valores são apenas divididos por 3, semelhante ao processo que seria feito através do `MinMaxScaler` do Scikit-Learn.

In [16]:
# Function definition to clean text
def clean_text(text: str) -> str:
    """
    Realiza limpeza textual baseada em expressões regulares.
    As transformações aplicadas são:
    - Normalização Unicode
    - Remoção de ruídos (como sequência de "===" ou " ---")
    - Remoção de HTML, URLs e E-mails
    - Remoção de caracteres não-principais e de controle
    - Normalização de espaços

    Params:
        text (str): Texto a ser limpo.

    Returns:
        str: Texto limpo.
    """
    if not text: return ""

    # Unicode normalization (NFKC)
    text = unicodedata.normalize('NFKC', text)

    # Remove noise (repeated sequences such as "=====" or "----")
    text = re.sub(r'([^a-zA-Z0-9\s])\1{2,}', ' ', text)

    # Remove HTML, URLs, E-mails
    text = re.sub(r'<.*?>', '', text) # HTML
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # URLs
    text = re.sub(r'\S+@\S+', '', text) # E-mails

    # Clean non-printable characters and control
    text = "".join(ch for ch in text if unicodedata.category(ch)[0] != "C")

    # Spaces normalization (useful for keeping cleaned chunks in RAG)
    text = re.sub(r'\s+', ' ', text).strip()

    return text.lower()

### Quati Dataset

In [17]:
# Clean Quati queries
def clean_quati_topics(batch):
    batch["clean_query"] = [clean_text(q) for q in batch["query"]]
    return batch

quati_topics = quati_topics.map(
    clean_quati_topics,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

quati_topics[:3]

{'query_id': [0, 1, 2],
 'query': ['qual a maior característica da flora brasileira?',
  'qual a maior característica da fauna brasileira?',
  'por que os países guiana e suriname não são filiados a conmebol?'],
 'clean_query': ['qual a maior característica da flora brasileira?',
  'qual a maior característica da fauna brasileira?',
  'por que os países guiana e suriname não são filiados a conmebol?']}

In [18]:
# Clean Quati passages
def clean_quati_passages(batch):
    batch["clean_passage"] = [clean_text(p) for p in batch["passage"]]
    return batch

quati_passages = quati_passages.map(
    clean_quati_passages,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

quati_passages["clean_passage"][:2]

['se você precisar de ajuda, visite o website nacional sobre a covid-19 ou ligue para a linha de apoio à covid-19 808 24 24 24 perguntas mais frequentes posso viajar entre sintra e cascais? quais são as restrições de viagem em cascais? qual o número de telefone de apoio para a covid 19 em cascais? preciso utilizar máscara facial no transporte público em cascais? a prática do distanciamento social é compulsória em cascais? o que eu devo fazer caso apresente sintomas da covid-19 quando chegar em cascais? última atualização: 25 abr 2022 aplicam-se exceções, para detalhes completos: european union. estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à covid-19. esta informação é compilada a partir de fontes oficiais. ao melhor de nosso conhecimento, está correta de acordo com a última atualização. visite avisos de viagem rome2rio para ajuda geral. perguntas & respostas qual a maneira mais econômica de ir de sintra para cascais? qual a maneira

In [19]:
# Normalize Quati scores
# Function to normalize scores
def normalize_quati_score(score):
    return score / 3.0

quati_qrels = quati_qrels.map(
    lambda x: {"score": normalize_quati_score(x["score"])},
    num_proc=cpu_count
)

quati_qrels[:3]

{'query_id': [1, 1, 1],
 'passage_id': ['clueweb22-pt0000-78-09747_0',
  'clueweb22-pt0000-96-07278_111',
  'clueweb22-pt0001-85-06153_3'],
 'score': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]}

### MS MARCO

In [20]:
# Clean MS MARCO queries
def clean_msmarco_queries(batch):
    batch["clean_query"] = [clean_text(q) for q in batch["query"]]
    return batch

msmarco = msmarco.map(
    clean_msmarco_queries,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)

msmarco.select_columns(["query", "clean_query"])[:3]

{'query': [')what was the immediate impact of the success of the manhattan project?',
  '_________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:',
  'why did stalin want control of eastern europe'],
 'clean_query': [')what was the immediate impact of the success of the manhattan project?',
  'justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:',
  'why did stalin want control of eastern europe']}

In [21]:
# Clean MS MARCO passages
def clean_msmarco_passages(batch):
    # This list will store dictionaries, each representing a query's passages and their selection status
    cleaned_passages = []

    for passages_dict_for_one_query in batch["passages"]:
        cleaned_passage_texts = [
            clean_text(p_text)
            for p_text in passages_dict_for_one_query["passage_text"]
        ]
        # Combine cleaned passages and their is_selected flags into a single dictionary per query
        cleaned_passages.append({
            "passage_text": cleaned_passage_texts,
            "is_selected": passages_dict_for_one_query["is_selected"]
        })

    # The 'clean_passages' column will now store this list of dictionaries
    # This effectively 'moves' the is_selected information into the clean_passages column's structure
    batch["clean_passages"] = cleaned_passages
    return batch


msmarco = msmarco.map(
    clean_msmarco_passages,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count
)


msmarco["clean_passages"][:3]

[{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'passage_text': ['the presence of communication amid scientific minds was equally important to the success of the manhattan project as scientific intellect was. the only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.',
   'the manhattan project and its atomic bomb helped bring an end to world war ii. its legacy of peaceful uses of atomic energy continues to have an impact on history and science.',
   'essay on the manhattan project - the manhattan project the manhattan project was to see if making an atomic bomb possible. the success of this project would forever change the world forever making it known that something this powerful can be manmade.',
   'the manhattan project was the name for a project conducted during world war ii, to develop the first atomic bomb. it refers specifically to the period of the pr

## Feature Engineering: Seleção das Features
Para treinamento do modelo de reranker regressivo, a estrutura proposta para o dataset é composta por um triplet no formato `(query, passage, score)`, sendo:
- `query`: A consulta realizada para busca.
- `passage`: O documento recuperado para consulta.
- `score`: Valor de relevância do documento para a consulta, normalizado entre 0 e 1.

Tendo isso em vista, faremos a seleção e organização de features de cada dataset para refletir a estrutura adequada.

### Quati

In [22]:
# Create dictionary lookup to get query by id
topics_lookup = {}
for example in quati_topics:
    topics_lookup[example["query_id"]] = example["clean_query"]

print(f"Created topics lookup dictionary with {len(topics_lookup)} entries.")
print("First 5 entries of topics_lookup:")
for i, (k, v) in enumerate(topics_lookup.items()):
    if i >= 5: break
    print(f"{k}: {v}")

Created topics lookup dictionary with 200 entries.
First 5 entries of topics_lookup:
0: qual a maior característica da flora brasileira?
1: qual a maior característica da fauna brasileira?
2: por que os países guiana e suriname não são filiados a conmebol?
3: por que a bolívia tem duas capitais?
4: qual o que significa decolonialidade?


In [23]:
# Create dictionary lookup to find passages by id
passages_lookup = {}
for example in quati_passages:
    passages_lookup[example["passage_id"]] = example["clean_passage"]

print(f"Created passages lookup dictionary with {len(passages_lookup)} entries.")
print("First 5 entries of passages_lookup:")
for i, (k, v) in enumerate(passages_lookup.items()):
    if i >= 5: break
    print(f"{k}: {v}")

Created passages lookup dictionary with 1000000 entries.
First 5 entries of passages_lookup:
clueweb22-pt0000-00-00003_1: se você precisar de ajuda, visite o website nacional sobre a covid-19 ou ligue para a linha de apoio à covid-19 808 24 24 24 perguntas mais frequentes posso viajar entre sintra e cascais? quais são as restrições de viagem em cascais? qual o número de telefone de apoio para a covid 19 em cascais? preciso utilizar máscara facial no transporte público em cascais? a prática do distanciamento social é compulsória em cascais? o que eu devo fazer caso apresente sintomas da covid-19 quando chegar em cascais? última atualização: 25 abr 2022 aplicam-se exceções, para detalhes completos: european union. estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à covid-19. esta informação é compilada a partir de fontes oficiais. ao melhor de nosso conhecimento, está correta de acordo com a última atualização. visite avisos de viagem rom

In [24]:
# Format quati dataset in triplet structure
# Function to map quati with the correct format in batches
def map_quati_data(batch):
    queries = [topics_lookup[query_id] for query_id in batch["query_id"]]
    passages = [passages_lookup[passage_id] for passage_id in batch["passage_id"]]
    scores = [score for score in batch["score"]]
    return {"query": queries, "passage": passages, "score": scores}

# Got quati formatted
quati_formatted = quati_qrels.map(
    map_quati_data,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count,
    remove_columns=quati_qrels.column_names
)

print(quati_formatted)
print("-" * 100)
quati_formatted[:5]

Map (num_proc=2):   0%|          | 0/1933 [00:00<?, ? examples/s]

Dataset({
    features: ['score', 'query', 'passage'],
    num_rows: 1933
})
----------------------------------------------------------------------------------------------------


{'score': [0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.6666666666666666,
  0.0],
 'query': ['qual a maior característica da fauna brasileira?',
  'qual a maior característica da fauna brasileira?',
  'qual a maior característica da fauna brasileira?',
  'qual a maior característica da fauna brasileira?',
  'qual a maior característica da fauna brasileira?'],
 'passage': ['onça-pintada - escola kids qual matéria está procurando ? ciências home ciências onça-pintada onça-pintada que tal conhecer um pouco mais a respeito da incrível onça-pintada? neste texto falaremos sobre o maior felino encontrado nas américas. conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. a onça-pintada é um dos símbolos da fauna brasileira a onça-pintada é um dos símbolos da fauna do brasil e o maior felino encontrado nas américas. esse animal é um mamífero da ordem carnivora e seu nome científico é panthera onca. leia também: classifica

### MS MARCO

In [25]:
# Function to format msmarco
def format_msmarco(batch):
    """
    Format a estrutura do MS MARCO para o formato (query, passage, score).

    Params:
        batch (dict): Batch de dados do MS MARCO.

    Returns:
        dict: Batch formatado.
    """
    queries = []
    passages = []
    scores = []

    # Iterate clean_query and the dictionary containing cleaned passages and scores for that query
    for query_text, passage_info_dict in zip(
        batch["clean_query"],
        batch["clean_passages"]
    ):
        # Iterate over the individual passages and their corresponding scores
        for p_text, is_selected_score in zip(
            passage_info_dict["passage_text"], # This is a list of passage strings
            passage_info_dict["is_selected"]   # This is a list of scores (0 or 1)
        ):
            queries.append(query_text)
            passages.append(p_text)
            scores.append(float(is_selected_score))

    return {"query": queries, "passage": passages, "score": scores}

In [26]:
# Get ms marco with the correct format
msmarco_formatted = msmarco.map(
    format_msmarco,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_count,
    remove_columns=["passages", "query", "answers", "wellFormedAnswers", "query_type", "query_id", "clean_query", "clean_passages"]
)

print(msmarco_formatted)
print("-" * 100)
msmarco_formatted[:5]

Dataset({
    features: ['query', 'passage', 'score'],
    num_rows: 8069749
})
----------------------------------------------------------------------------------------------------


{'query': [')what was the immediate impact of the success of the manhattan project?',
  ')what was the immediate impact of the success of the manhattan project?',
  ')what was the immediate impact of the success of the manhattan project?',
  ')what was the immediate impact of the success of the manhattan project?',
  ')what was the immediate impact of the success of the manhattan project?'],
 'passage': ['the presence of communication amid scientific minds was equally important to the success of the manhattan project as scientific intellect was. the only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.',
  'the manhattan project and its atomic bomb helped bring an end to world war ii. its legacy of peaceful uses of atomic energy continues to have an impact on history and science.',
  'essay on the manhattan project - the manhattan project the manhattan project wa

## Persistência dos dados processados

In [27]:
# Export Quati formatted
quati_formatted.to_parquet("./quati_processed.parquet")

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

2349304

In [28]:
# Export MS MARCO formatted
msmarco_formatted.to_parquet("./msmarco_processed.parquet")

Creating parquet from Arrow format:   0%|          | 0/8070 [00:00<?, ?ba/s]

3124636378